In [102]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import joblib

# Carregar o dataset
df = pd.read_excel('ready_for_filtering.xlsx')

# Lista de possíveis valores para agile_methods e metrics_category
agile_methods_list = ['Scrum', 'Kanban', 'ScrumBan', 'XP', 'Safe', 'Lean'] 
metrics_category_list = ['Cronograma e progresso', 'Produto', 'Processo', 'Tecnologia', 'Pessoas', 'Cliente']

# Nova lista de features com colunas expandidas
new_agile_columns = [f'agile_methods_{method.lower()}' for method in agile_methods_list]
new_metrics_columns = [f'metrics_category_{category.lower().replace(" ", "_")}' for category in metrics_category_list]

# Atualizar a lista de features
features = ['role', 'years_exp', 'org_size', 
            'use_metrics_planning', 'use_metrics_review', 
            'use_metrics_weekly', 'use_metrics_daily', 
            'use_metrics_retro'] + new_agile_columns + new_metrics_columns

# Preencher valores nulos com string vazia
df[features] = df[features].fillna('')

# Converter as colunas binárias "Sim"/"Não" para 1/0
df[features] = df[features].replace({'Sim': 1, 'Não': 0})

# Definir pesos ajustados para amplificar o impacto das colunas binárias e importantes
feature_weights = {
    'role': 4.5,  # Peso menor para 'role'
    'years_exp': 2.5,
    'org_size': 1.5,
    'use_metrics_planning': 6.5,  # Amplificação do peso para estas colunas
    'use_metrics_review': 6.5,
    'use_metrics_weekly': 6.5,
    'use_metrics_daily': 6.5,
    'use_metrics_retro': 6.5,
    'agile_methods_scrum': 7.0,  # Peso ainda maior para métodos ágeis
    'agile_methods_kanban': 7.0,
    'agile_methods_scrumban': 7.0,
    'agile_methods_lean': 7.0,
    'agile_methods_xp': 7.0,
    'agile_methods_safe': 7.0,
    'metrics_category_cronograma_e_progresso': 3.5,  # Amplificação do peso para categorias de métricas
    'metrics_category_produto': 3.5,
    'metrics_category_processo': 3.5,
    'metrics_category_tecnologia': 3.5,
    'metrics_category_pessoas': 3.5,
    'metrics_category_cliente': 3.5
}

# Gerar vetores TF-IDF apenas para as colunas não binárias (texto)
tfidf_vectorizers = {}
tfidf_matrices = {}
for feature in features:
    if df[feature].dtype == 'object':  # Aplicar TF-IDF apenas em colunas de texto
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrices[feature] = tfidf_vectorizer.fit_transform(df[feature])
        tfidf_vectorizers[feature] = tfidf_vectorizer  # Salvar o vectorizer usado
    else:
        # Para colunas binárias, convertemos diretamente em matriz numérica
        tfidf_matrices[feature] = df[[feature]].values


model_path = 'collaborative_filtering_model.joblib'

# Criar um dicionário com todos os elementos necessários para o modelo
model_data = {
    'tfidf_matrices': tfidf_matrices,  # Inclui as matrizes TF-IDF
    'feature_weights': feature_weights,
    'df': df,
    'features': features,
    'tfidf_vectorizers': tfidf_vectorizers  # Inclui os vectorizers
}

# Salvar o dicionário com os dados do modelo usando joblib
joblib.dump(model_data, model_path)

print(f"Modelo salvo em: {model_path}")

Modelo salvo em: collaborative_filtering_model.joblib


/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_3758/4113872536.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[features] = df[features].replace({'Sim': 1, 'Não': 0})
